In [ ]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
#include <time.h>

//# funkcja potęgująca bo funkcja w math.h pow nie działa 

double potegowanie(double a , int b )
{
      double result=1.0;
      for (int i = 1; i <= b; i++)
      {
          result *= a;
      }
      return result; 
}

int main(int argc, char **argv)
{
    int numer_procesu;
    int liczba_procesow;
    int tag = 1;
    
    MPI_Status status;   //#Struktura, która jest używana do uzyskania informacji o statusie operacji MPI_Recv.

    MPI_Init(&argc, &argv);           //#MPI_Init: Inicjalizuje środowisko MPI i zapewnia wywołanie wszystkich innych funkcji MP

    MPI_Comm_rank(MPI_COMM_WORLD, &numer_procesu);  //#Określa identyfikator (numer) bieżącego procesu MPI. Funkcja ta zwraca identyfikator bieżącego procesu za pośrednictwem zmiennej numer_procesu.

    MPI_Comm_size(MPI_COMM_WORLD, &liczba_procesow);    //#Zwraca liczbę procesów MPI będących częścią komunikacji. Funkcja ta zwraca liczbę procesów MPI za pośrednictwem zmiennej liczba_procesów.
    
    clock_t t;
    double time_taken;

    if(numer_procesu == 0)
    {
      double lpi = 0 ,x = 0,total_time =0,t=0;

      for(int i=1; i<liczba_procesow; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);  //#Odbiera dane z innego procesu MPI. Funkcja ta odbiera dane od procesu o numerze różnym od 0 i zapisuje je w zmiennej x.
        lpi +=x;
        MPI_Recv(&t, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        total_time+=t;
      }
      printf("Wartosc liczby pi = %f \n", 4 * lpi);
      printf("Total time taken = %f \n", total_time);
    }

    if(numer_procesu != 0)
    {
      t = clock();  //#Zmienna t jest używana do mierzenia czasu, który został wykorzystany przez proces, aby wykonać jego zadanie.
      //#Jest ona inicjalizowana za pomocą funkcji clock(), która zwraca ilość taktów procesora, które zostały użyte od czasu uruchomienia programu.

      double x = potegowanie( -1, numer_procesu - 1) / ( 2 * numer_procesu - 1); 
      printf("Wynik procesu %d = %f \t" , numer_procesu , x);
      MPI_Send(&x, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);  //#Wysyła dane z jednego procesu MPI do innego procesu MPI. Funkcja ta wysyła zmienną x z bieżącego procesu do procesu o numerze 0.

      t = clock() - t;  //#zadania t jest ponownie ustawiana za pomocą clock() i odejmowana od t w celu uzyskania ilości taktów procesora, które zostały użyte na wykonanie zadania.

      time_taken = ((double)t)/CLOCKS_PER_SEC; //#Zmienna time_taken jest używana do przechowywania czasu w sekundach, który został wykorzystany na wykonanie zadania przez proces.
      //#Jest to obliczone przez podzielenie t przez CLOCKS_PER_SEC, które jest stałą, która określa ilość taktów procesora na

      MPI_Send(&time_taken, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      printf("Czas wykonywania sie procesu nr %d = %f\n",numer_procesu,time_taken);
    }
    MPI_Finalize(); //#Zamyka środowisko MPI i zwalnia zasoby z niego używane.
}
EOF

mpicc pi-mpi.c && mpirun -n 1 --allow-run-as-root a.out 

//#program zostanie uruchomiony z jednym procesem (-n 1). Wynik działania programu będzie wyświetlony na standardowym wyjściu.